In [1]:
import sys
import os
import random
import collections
import numpy as np
import lightgbm as lgb
import pandas as pd


In [2]:
"""
GBDT已经训练好了，我们需要GBDT输出的叶子节点作为输入数据X传给FM，一共30个叶子节点（32颗树 如何利用输出的结果）

那么输入给FM的数据格式就是X中不是0的数据的index:value。（零值对于训练不起作用， Wij需要两个都是非零值数据量更小，所以对于FM公式化简）
val是树的节点index
一段真实数据如下：0 0:31 1:61 2:93 3:108 4:149 5:182 6:212 7:242 8:277 9:310 10:334 11:365 12:401 13:434 14:465 15:491 16:527 17:552 18:589 19:619 20:648 21:678 22:697 23:744 24:770 25:806 26:826 27:862 28:899 29:928 30:955 31:988
"""

import pickle
with open("/media/data/cuixuange/Criteo_dataset/model/gbm_dump.p", 'rb') as pickle_file:
    gradientBoostModel,parameter_dump = pickle.load(pickle_file)
print(type(gradientBoostModel))
print(type(parameter_dump))

#  # gbdt_model中的参数信息？？ 
print(len(parameter_dump["tree_info"]))   #32棵树即 32轮基学习器的迭代次数
print(parameter_dump["tree_info"][1])


<class 'lightgbm.basic.Booster'>
<class 'dict'>
32
{'tree_index': 1, 'num_leaves': 30, 'num_cat': 15, 'shrinkage': 0.05, 'tree_structure': {'split_index': 0, 'split_feature': 0, 'split_gain': 25220, 'threshold': 1.0000000180025095e-35, 'decision_type': '<=', 'default_left': True, 'missing_type': 'None', 'internal_value': 0, 'internal_count': 719992, 'left_child': {'split_index': 2, 'split_feature': 6, 'split_gain': 7863.419921875, 'threshold': 0.025000000000000005, 'decision_type': '<=', 'default_left': True, 'missing_type': 'None', 'internal_value': -0.295885, 'internal_count': 491927, 'left_child': {'split_index': 6, 'split_feature': 30, 'split_gain': 2139.830078125, 'threshold': '40||60||80||91||101||150||181||185||191||208||209||211||231||238||244||250||280||288||326||334||344||351||356||364||378||383||391||400||407||477||574', 'decision_type': '==', 'default_left': False, 'missing_type': 'Zero', 'internal_value': -0.531481, 'internal_count': 298988, 'left_child': {'leaf_index': 0,

In [3]:
def generat_lgb2fm_data(outdir, gbm, dump, tr_path, va_path, te_path, _sep = '\t'):
    """
gbmodel paramter_dump tr_path va_path te_path都是输入的数据
    """
    """
设置参数pred_leaf=true的作用
the output will be a matrix of (nsample, ntrees) with each record indicating the predicted leaf index of each sample in each tree
    """
    with open(os.path.join(outdir, 'train_lgb2fm.txt'), 'w') as out_train:
        with open(os.path.join(outdir, 'valid_lgb2fm.txt'), 'w') as out_valid:
            with open(os.path.join(outdir, 'test_lgb2fm.txt'), 'w') as out_test:
                df_train_ = pd.read_csv(tr_path, header=None, sep=_sep)
                df_valid_ = pd.read_csv(va_path, header=None, sep=_sep)
                df_test_= pd.read_csv(te_path, header=None, sep=_sep)

                y_train_ = df_train_[0].values
                y_valid_ = df_valid_[0].values                

                X_train_ = df_train_.drop(0, axis=1).values
                X_valid_ = df_valid_.drop(0, axis=1).values
                X_test_= df_test_.values
                
                train_leaves= gbm.predict(X_train_, num_iteration=gbm.best_iteration, pred_leaf=True)
                valid_leaves= gbm.predict(X_valid_, num_iteration=gbm.best_iteration, pred_leaf=True)
                print(valid_leaves.shape)
                test_leaves= gbm.predict(X_test_, num_iteration=gbm.best_iteration, pred_leaf=True)

                tree_info = dump['tree_info']
                tree_counts = len(tree_info)
                for i in range(tree_counts):
                    train_leaves[:, i] = train_leaves[:, i] + tree_info[i]['num_leaves'] * i + 1
                    
    ######每次基学习器输出的就是一个one-hot变量 只有一个节点是有值的
#                     print(valid_leaves[0, i],valid_leaves[0, i],tree_info[i]['num_leaves'] * i + 1)
                    valid_leaves[:, i] = valid_leaves[:, i] + tree_info[i]['num_leaves'] * i + 1
                    
                    test_leaves[:, i] = test_leaves[:, i] + tree_info[i]['num_leaves'] * i + 1
#                     print(train_leaves[:, i])
#                     print(tree_info[i]['num_leaves'])





#                 print(valid_leaves.shape)
                for idx in range(len(y_train_)):            
                    out_train.write((str(y_train_[idx]) + '\t'))  #label
                    out_train.write('\t'.join(
                        ['{}:{}'.format(ii, val) for ii,val in enumerate(train_leaves[idx]) if float(val) != 0 ]) + '\n')
                    
                for idx in range(len(y_valid_)):                   
                    out_valid.write((str(y_valid_[idx]) + '\t'))
                    out_valid.write('\t'.join(
                        ['{}:{}'.format(ii, val) for ii,val in enumerate(valid_leaves[idx]) if float(val) != 0 ]) + '\n')
                    
                for idx in range(len(X_test_)):                   
                    out_test.write('\t'.join(
                        ['{}:{}'.format(ii, val) for ii,val in enumerate(test_leaves[idx]) if float(val) != 0 ]) + '\n')

In [4]:
preprocess_data_dir = "/media/data/cuixuange/Criteo_dataset/preprocess_data/"
generat_lgb2fm_data(preprocess_data_dir, gradientBoostModel, parameter_dump, preprocess_data_dir+'train_lgb.txt',
                    preprocess_data_dir+'valid_lgb.txt', preprocess_data_dir+'test_lgb.txt', '\t')

print(gradientBoostModel.best_iteration)

(100009, 32)
32


In [5]:
#fm 训练

libfmDir = "/media/data/cuixuange/Criteo_dataset/Origincode_and_bin/kaggle_criteo_ctr_challenge-/libfm/libfm/"
cmd = libfmDir + '/bin/libFM -task c -train '+ preprocess_data_dir + 'train_lgb2fm.txt -test '+ preprocess_data_dir + \
'/valid_lgb2fm.txt -dim ’1,1,8’ -iter 64 -method sgd -learn_rate 0.00000001 -regular ’0,0,0.01’ -init_stdev 0.1 -save_model fm_model'

print(cmd)
os.popen(cmd).readlines()

/media/data/cuixuange/Criteo_dataset/Origincode_and_bin/kaggle_criteo_ctr_challenge-/libfm/libfm//bin/libFM -task c -train /media/data/cuixuange/Criteo_dataset/preprocess_data/train_lgb2fm.txt -test /media/data/cuixuange/Criteo_dataset/preprocess_data//valid_lgb2fm.txt -dim ’1,1,8’ -iter 64 -method sgd -learn_rate 0.00000001 -regular ’0,0,0.01’ -init_stdev 0.1 -save_model fm_model


['----------------------------------------------------------------------------\n',
 'libFM\n',
 '  Version: 1.4.4\n',
 '  Author:  Steffen Rendle, srendle@libfm.org\n',
 '  WWW:     http://www.libfm.org/\n',
 'This program comes with ABSOLUTELY NO WARRANTY; for details see license.txt.\n',
 'This is free software, and you are welcome to redistribute it under certain\n',
 'conditions; for details see license.txt.\n',
 '----------------------------------------------------------------------------\n',
 'Loading train...\t\n',
 'has x = 1\n',
 'has xt = 0\n',
 'num_rows=899991\tnum_values=28799712\tnum_features=32\tmin_target=0\tmax_target=1\n',
 'Loading test... \t\n',
 'has x = 1\n',
 'has xt = 0\n',
 'num_rows=100009\tnum_values=3200288\tnum_features=32\tmin_target=0\tmax_target=1\n',
 '#relations: 0\n',
 'Loading meta data...\t\n',
 'learnrate=1e-08\n',
 'learnrates=1e-08,1e-08,1e-08\n',
 '#iterations=64\n',
 "SGD: DON'T FORGET TO SHUFFLE THE ROWS IN TRAINING DATA TO GET THE BEST RESULT

In [ ]:
#fm get result
# 我们把训练、验证和测试数据输入给FM，得到FM层的输出，输出的文件名为*.fm.logits

libfmDir = "/media/data/cuixuange/Criteo_dataset/Origincode_and_bin/kaggle_criteo_ctr_challenge-/libfm/libfm/"
cmd = libfmDir + '/bin/libFM -task c -train '+ preprocess_data_dir + 'train_lgb2fm.txt -test '+ preprocess_data_dir + \
'/valid_lgb2fm.txt -dim ’1,1,8’ -iter 32 -method sgd -learn_rate 0.00000001 -regular ’0,0,0.01’ -init_stdev 0.1 '+ \
'-load_model fm_model -train_off true -prefix tr'

print(cmd)
os.popen(cmd).readlines()

In [25]:
libfmDir = "/media/data/cuixuange/Criteo_dataset/Origincode_and_bin/kaggle_criteo_ctr_challenge-/libfm/libfm/"
cmd = libfmDir + '/bin/libFM -task c -train '+ preprocess_data_dir + 'valid_lgb2fm.txt -test '+ preprocess_data_dir + \
'/valid_lgb2fm.txt -dim ’1,1,8’ -iter 32 -method sgd -learn_rate 0.00000001 -regular ’0,0,0.01’ -init_stdev 0.1 '+ \
'-load_model fm_model -train_off true -prefix va'

print(cmd)
os.popen(cmd).readlines()


/media/data/cuixuange/Criteo_dataset/Origincode_and_bin/kaggle_criteo_ctr_challenge-/libfm/libfm//bin/libFM -task c -train /media/data/cuixuange/Criteo_dataset/preprocess_data/valid_lgb2fm.txt -test /media/data/cuixuange/Criteo_dataset/preprocess_data//valid_lgb2fm.txt -dim ’1,1,8’ -iter 32 -method sgd -learn_rate 0.00000001 -regular ’0,0,0.01’ -init_stdev 0.1 -load_model fm_model -train_off true -prefix va


['----------------------------------------------------------------------------\n',
 'libFM\n',
 '  Version: 1.4.4\n',
 '  Author:  Steffen Rendle, srendle@libfm.org\n',
 '  WWW:     http://www.libfm.org/\n',
 'This program comes with ABSOLUTELY NO WARRANTY; for details see license.txt.\n',
 'This is free software, and you are welcome to redistribute it under certain\n',
 'conditions; for details see license.txt.\n',
 '----------------------------------------------------------------------------\n',
 'Loading train...\t\n',
 'has x = 1\n',
 'has xt = 0\n',
 'num_rows=100009\tnum_values=3156842\tnum_features=32\tmin_target=0\tmax_target=1\n',
 'Loading test... \t\n',
 'has x = 1\n',
 'has xt = 0\n',
 'num_rows=100009\tnum_values=3156842\tnum_features=32\tmin_target=0\tmax_target=1\n',
 '#relations: 0\n',
 'Loading meta data...\t\n',
 'Reading FM model... \t\n',
 'Final\tTrain=0.255347\tTest=0.255347\n']

In [ ]:
libfmDir = "/media/data/cuixuange/Criteo_dataset/Origincode_and_bin/kaggle_criteo_ctr_challenge-/libfm/libfm/"
cmd = libfmDir + '/bin/libFM -task c -train '+ preprocess_data_dir + 'test_lgb2fm.txt -test '+ preprocess_data_dir + \
'/valid_lgb2fm.txt -dim ’1,1,8’ -iter 32 -method sgd -learn_rate 0.00000001 -regular ’0,0,0.01’ -init_stdev 0.1 '+ \
'-load_model fm_model -train_off true -prefix te -test2predict true'

print(cmd)
os.popen(cmd).readlines()